Spectral shape descriptors and functional maps 

In [7]:
import pyFM
from pyFM import mesh
from pyFM import functional
from pyFM import eval


from shape_utils.spectral_descr import calculate_descriptors
from shape_utils.utils import find_minimum_distance_meshes
from shape_utils.functional_maps import calculate_functional_maps, compute_shape_difference
from shape_utils.similarity_scores import calculate_geodesic_norm_score
from analysis_tools.visualisation_correspondence import visualize_correspondence_meshplot
from analysis_tools.superposition_shapes import calculate_rotation_translation
import os
import seaborn as sns
import numpy as np
import meshplot as mp
from scipy.linalg import norm
import matplotlib.pyplot as plt
import csv



In [3]:
#Functions for ploting correspondace and 3D spectral maps
def plot_mesh(myMesh,cmap=None):
   mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist,myMesh1.facelist, c=cmap1[:,1],s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2[:,1], s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap



In [4]:
def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

Calculate correspondace matrix, point to point map and disimilarity score for two input files with the surface meshes (.off/.obj)

In [ ]:

file_mesh1 = '/Users/gdiazleines/results/shape-retrieval/benchmarking/Spike_protein/meshes/meshes_low_quality/fixed_meshes/7V7S.obj'
file_mesh2 = '/Users/gdiazleines/results/shape-retrieval/benchmarking/Spike_protein/meshes/meshes_low_quality/fixed_meshes/7V7D.obj'

mesh1 = mesh.TriMesh(file_mesh1, area_normalize=True, center=False)
mesh2 = mesh.TriMesh(file_mesh2, area_normalize=True, center=False)

model = functional.FunctionalMapping(mesh1,mesh2) 

neigvecs = 200
ndescr = 100
step = 1
n_ev = 50
output = './example_data/'
descr = 'WKS'
landmarks = None

descr1,descr2 = calculate_descriptors(model,neigvecs,n_ev,ndescr,step,landmarks,output,descr)
print(np.shape(descr1))
p2p_21,FM = calculate_functional_maps(model,n_cpus=8,refine=None)
sim_score = calculate_geodesic_norm_score(FM)
print('disimilarity score:', sim_score)


Plot correspondace matrix and 3D maps

In [ ]:
#cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
#double_plot(mesh1,mesh2,cmap1,cmap2)
fig, (ax1) = plt.subplots(1, figsize=(3, 3))
sns.heatmap(FM,ax=ax1,vmax=1.0,vmin=-1.0,cmap='seismic')

Calculate disimilarity score and plot correspondace matrix from output file *_FM.csv

In [ ]:
file_FM ='/Users/gdiazleines/results/shape-retrieval/benchmarking/Spike_protein/analysis/molstar_low-resolution/example/7V7S_7V7D_FM.csv'


with open(file_FM) as csvfile:
    FM = list(csv.reader(csvfile))
    FM = np.asarray(FM, dtype=float)

#calculate disimilarity score
sim_score = calculate_geodesic_norm_score(FM)
print('disimilarity score:', sim_score)

#plot correspondance matrix
fig, (ax1) = plt.subplots(1, figsize=(5, 5))
sns.heatmap(FM,ax=ax1,vmax=1.0,vmin=-1.0,cmap='seismic')

In [ ]:
source_path=file_mesh1 
target_path=file_mesh2

map_path = '/Users/gdiazleines/results/shape-retrieval/benchmarking/Spike_protein/analysis/molstar_low-resolution/example/7V7S_7V7D_p2p21.csv'

R,T=calculate_rotation_translation(source_path,target_path, map_path)
t = T.flatten()
visualize_correspondence_meshplot(source_path, target_path, map_path, R)

Zernike Descriptors

In [16]:
from shape_utils.zernike_descr import get_inv
from shape_utils.meshes import remove_until_vertex
from shape_utils.similarity_scores import predict_similarity_zernike
import itertools

In [ ]:

file_entries='/Users/gdiazleines/results/shape-retrieval/benchmarking/bacterial_ribosome/shape_analysis/list_entries.txt'
with open(file_entries, 'r') as file:
    # Read all lines, stripping the newline character
    entry_list = [line.strip() for line in file]
print(np.shape(entry_list))

for entry in entry_list:
    mesh_file_1='{}.obj'.format(entry)

    print(mesh_file_1)

    file_mesh_obj = '/Users/gdiazleines/results/shape-retrieval/benchmarking/bacterial_ribosome/meshes/fixed_collapse_0.5/{}'.format(mesh_file_1)
    
    #removing extra lines in .obj file (obj2grid has trouble to read extra header lines in .obj files)
    remove_until_vertex(file_mesh_obj)
    
    #remove_until_vertex(file_mesh1_obj)
    output = '/Users/gdiazleines/results/shape-retrieval/benchmarking/bacterial_ribosome/shape_analysis/zernike_data'
    
    get_inv(file_mesh_obj,entry,'map2zernike', 'obj2grid',output)
    
                

In [ ]:
   
input_dir = '/Users/gdiazleines/results/shape-retrieval/benchmarking/bacterial_ribosome/shape_analysis/zernike_data'
output_dir ='/Users/gdiazleines/results/shape-retrieval/benchmarking/bacterial_ribosome/shape_analysis/zernike_data'
predict_similarity_zernike(input_dir,output_dir)